In [ ]:
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 11.1 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
# import bitsandbytes as bnb

In [ ]:
from huggingface_hub import login
login(new_session=False)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
model_name = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
        model_name,
        load_in_4bit=True,
        # quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
    )

tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): Mist

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer = tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

Device set to use cuda:0


In [ ]:
prompt = "Complete the following mathematical operation and provide its output. What is the value of 3x8"

sequences = pipe(
    prompt,
    do_sample=True,
    max_new_tokens=100,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,
)
print(sequences[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Complete the following mathematical operation and provide its output. What is the value of 3x8?

### Solution

Given,

`3x8`

`3 x 8 = 24`

Therefore, the value of 3x8 is 24


In [ ]:
model_name = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
        model_name,
        load_in_4bit=True,
        # quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
    )

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer = tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

prompt = "For each Korean sentence below, provide an exact English translation. Do not paraphrase or interpret. Preserve word order and punctuation as much as possible. \
Output only the translation for each sentence.Korean Sentences:\
    1. 아내가 외출하기 전 끓여놓은 국과 반찬을 식판에 떠서 먹이기만 하면 되는데 말처럼 쉬운 일이 아니다."
sequences = pipe(
    prompt,
    do_sample=True,
    max_new_tokens=100,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,
)
print(sequences[0]['generated_text'])
'''
Output:
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
For each Korean sentence below, provide an exact English translation. Do not paraphrase or interpret. Preserve word order and punctuation as much as possible. Output only the translation for each sentence.Korean Sentences:
    1. 아내가 외출하기 전 끓여놓은 국과 반찬을 식판에 떠서 먹이기만 하면 되는데 말처럼 쉬운 일이 아니다.
    2. 밤이 새벽에 붙어들면 사람들은 잠이 오던 잠이 오던 힘들게 잠자는 것 같다.
    3. 잠이 오던 잠이 오던 사람들이 잠을
'''

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


For each Korean sentence below, provide an exact English translation. Do not paraphrase or interpret. Preserve word order and punctuation as much as possible. Output only the translation for each sentence.Korean Sentences:    1. 아내가 외출하기 전 끓여놓은 국과 반찬을 식판에 떠서 먹이기만 하면 되는데 말처럼 쉬운 일이 아니다.
    2. 밤이 새벽에 붙어들면 사람들은 잠이 오던 잠이 오던 힘들게 잠자는 것 같다.
    3. 잠이 오던 잠이 오던 사람들이 잠을 


In [ ]:
prompt = "Translate this sentence to Korean: \"How are you today?\""

sequences = pipe(
    prompt,
    do_sample=True,
    max_new_tokens=100,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,
)
print(sequences[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Translate this sentence to Korean: "How are you today?"

You've probably noticed that this is an example sentence in this lesson.

There are many English sentences that can be translated to Korean as "How are you today?". For example, "How are you doing today?", "How are you today?", "How are you?", "How is it today?", "How is today?", and so on.

A student asked me, "Is it okay if I say 'How are you today?' instead


In [ ]:
prompt = ''' Given a set of korean sentences, translate only those sentences to english. Do not provide any additional information or add new values. The first one is given as an example and should not be translated.
[
  {
    "korea": "홍콩은 중국의 특별行政區입니다.",
    "english": "Hong Kong is a special administrative region of China."
  },
  {
    "korea": "한국은 동아시아의 한국민들이 거주하고 있는 나라입니다."
  }'''

sequences = pipe(
    prompt,
    do_sample=True,
    max_new_tokens=1000,
    temperature=0.05,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,
)
print(sequences[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Given a set of korean sentences, translate only those sentences to english. Do not provide any additional information or add new values. The first one is given as an example and should not be translated.
[
  {
    "korea": "홍콩은 중국의 특별行政區입니다.",
    "english": "Hong Kong is a special administrative region of China."
  },
  {
    "korea": "한국은 동아시아의 한국민들이 거주하고 있는 나라입니다."
  }
]
















































































































































































































































































































































































































































































































































































































































In [ ]:
sequences

[{'generated_text': "Answer exactly in only one sentence. Translate the following Korean text to english. '''아내가 외출하기 전 끓여놓은 국과 반찬을 식판에 떠서 먹이기만 하면 되는데 말처럼 쉬운 일이 아니다'''\n\nAsked on 15 April 2022 by yosin.\n\nAsked on 15 April 2022 by yosin.\n\nI want to read the following Korean sentence. I am not sure about the meaning of this sentence.\n\nAsked on 23 March 2022 by joseph23.\n\nAsked on 23 March 2022 by"}]